In [195]:
import h5py
import os
import numpy as np

from ramandecompy import dataprep

In [196]:
os.remove('string_test.hdf5')
dataprep.new_hdf5('string_test')

In [197]:
hdf5 = h5py.File('string_test.hdf5', 'r+')

In [198]:
hdf5['dataset'] = [1, 2, 3, 4, 5, 6, 7]
hdf5.close()

In [199]:
dataprep.view_hdf5('string_test.hdf5')

**** string_test.hdf5 ****
dataset


In [200]:
def create_dataset(h5py_file):
    data = (1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 'label')
    my_datatype = np.dtype([('fraction', np.float),
                        ('center', np.float),
                        ('sigma', np.float),
                        ('amplitude', np.float),
                        ('fwhm', np.float),
                        ('height', np.float),
                        ('area under the curve', np.float),
                        ('label', h5py.special_dtype(vlen=str))])
    dataset = h5py_file.create_dataset('label_added', (len(data),), dtype=my_datatype)
    data_array = np.array(data, dtype=my_datatype)
    dataset = data_array

In [201]:
hdf5 = h5py.File('string_test.hdf5', 'r+')
data = (1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 'label')
my_datatype = np.dtype([('fraction', np.float),
                    ('center', np.float),
                    ('sigma', np.float),
                    ('amplitude', np.float),
                    ('fwhm', np.float),
                    ('height', np.float),
                    ('area under the curve', np.float),
                    ('label', h5py.special_dtype(vlen=str))])
dataset = hdf5.create_dataset('label_added', (1,), dtype=my_datatype)
data_array = np.array(data, dtype=my_datatype)

In [202]:
dataprep.view_hdf5('string_test.hdf5')

**** string_test.hdf5 ****
dataset
label_added


In [208]:
list(hdf5['label_added'])

[(1., 2., 3., 4., 5., 6., 7., 'label')]

In [204]:
dataset[...] = data_array

In [205]:
list(hdf5['label_added'])

[(1., 2., 3., 4., 5., 6., 7., 'label')]

In [206]:
foo = list(hdf5['label_added'])
foo[0][7]

'label'